In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

@tf.keras.utils.register_keras_serializable()
class ClassToken(Layer):
    def __init__(self, **kwargs):  # Add **kwargs here
        super().__init__(**kwargs)  # Pass **kwargs to super().__init__

    def build(self, input_shape):
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            initial_value = w_init(shape=(1, 1, input_shape[-1]), dtype=tf.float32),
            trainable = True
        )

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        hidden_dim = self.w.shape[-1]

        cls = tf.broadcast_to(self.w, [batch_size, 1, hidden_dim])
        cls = tf.cast(cls, dtype=inputs.dtype)
        return cls

# @tf.keras.utils.register_keras_serializable()
# class ClassToken(Layer):
#     def __init__(self):
#         super().__init__()

#     def build(self, input_shape):
#         w_init = tf.random_normal_initializer()
#         self.w = tf.Variable(
#             initial_value = w_init(shape=(1, 1, input_shape[-1]), dtype=tf.float32),
#             trainable = True
#         )

#     def call(self, inputs):
#         batch_size = tf.shape(inputs)[0]
#         hidden_dim = self.w.shape[-1]

#         cls = tf.broadcast_to(self.w, [batch_size, 1, hidden_dim])
#         cls = tf.cast(cls, dtype=inputs.dtype)
#         return cls

def mlp(x, cf):
    x = Dense(cf["mlp_dim"], activation="gelu")(x)
    x = Dropout(cf["dropout_rate"])(x)
    x = Dense(cf["hidden_dim"])(x)
    x = Dropout(cf["dropout_rate"])(x)
    return x

def transformer_encoder(x, cf):
    skip_1 = x
    x = LayerNormalization()(x)
    x = MultiHeadAttention(
        num_heads=cf["num_heads"], key_dim=cf["hidden_dim"])(x, x)
    x = Add()([x, skip_1])

    skip_2 = x
    x = LayerNormalization()(x)
    x = mlp(x, cf)
    x = Add()([x, skip_2])

    return x

def ViT(cf):
    """ Inputs """
    input_shape = (cf["num_patches"], cf["patch_size"]*cf["patch_size"]*cf["num_channels"])
    inputs = Input(input_shape)

    """ Patch + Position Embeddings """
    patch_embed = Dense(cf["hidden_dim"])(inputs)

    positions = tf.range(start=0, limit=cf["num_patches"], delta=1)
    pos_embed = Embedding(input_dim=cf["num_patches"], output_dim=cf["hidden_dim"])(positions) ## (256, 768)
    embed = patch_embed + pos_embed ## (None, 256, 768)

    """ Adding Class Token """
    token = ClassToken()(embed)
    x = Concatenate(axis=1)([token, embed]) ## (None, 257, 768)#classsifer head
    #encoder is 26,1000

    for _ in range(cf["num_layers"]):
        x = transformer_encoder(x, cf)

    """ Classification Head """
    x = LayerNormalization()(x)     ## (None, 257, 768)
    x = x[:, 0, :]
    x = Dense(cf["num_classes"], activation="softmax")(x)

    model = Model(inputs, x)
    return model


if __name__ == "__main__":
    hp = {}
    hp["image_size"] = 200
    hp["num_channels"] = 3
    hp["patch_size"] = 25
    hp["num_patches"] = (hp["image_size"]**2) // (hp["patch_size"]**2)
    hp["flat_patches_shape"] = (hp["num_patches"], hp["patch_size"]*hp["patch_size"]*hp["num_channels"])

    hp["batch_size"] = 8
    hp["lr"] = 1e-4
    hp["num_epochs"] = 500
    hp["num_classes"] = 3
    hp["class_names"] = ["Benign", "Malignant", "Normal"]

    hp["num_layers"] = 12
    hp["hidden_dim"] = 768
    hp["mlp_dim"] = 2304
    hp["num_heads"] = 6
    hp["dropout_rate"] = 0.1

    model = ViT(hp)
    model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 64, 1875)       │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 64, 768)        │      1,440,768 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 64, 768)        │              0 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ class_token (ClassToken)  │ (None, 1, 768)         │              0 │ add[0][0]              │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 65, 768)        │              0 │ class_token[0][0],     │
│                           │                        │                │ add[0][0]              │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization       │ (None, 65, 768)        │          1,536 │ concatenate[0][0]      │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention      │ (None, 65, 768)        │     14,170,368 │ layer_normalization[0… │
│ (MultiHeadAttention)      │                        │                │ layer_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_1 (Add)               │ (None, 65, 768)        │              0 │ multi_head_attention[… │
│                           │                        │                │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_1     │ (None, 65, 768)        │          1,536 │ add_1[0][0]            │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 65, 2304)       │      1,771,776 │ layer_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 65, 2304)       │              0 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 65, 768)        │      1,770,240 │ dropout_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_2 (Dropout)       │ (None, 65, 768)        │              0 │ dense_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_2 (Add)               │ (None, 65, 768)        │              0 │ dropout_2[0][0],       │
│                           │                        │                │ add_1[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_2     │ (None, 65, 768)        │          1,536 │ add_2[0][0]            │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_1    │ (None, 65, 768)        │     14,170,368 │ layer_normalization_2… │
│ (MultiHeadAttention) 

 Total params: 214,030,083 (816.46 MB)

 Trainable params: 214,030,083 (816.46 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
 !pip install patchify

In [ ]:

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from patchify import patchify
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping

""" Hyperparameters """
hp = {}
hp["image_size"] = 200
hp["num_channels"] = 3
hp["patch_size"] = 25
hp["num_patches"] = (hp["image_size"]**2) // (hp["patch_size"]**2)
hp["flat_patches_shape"] = (hp["num_patches"], hp["patch_size"]*hp["patch_size"]*hp["num_channels"])

hp["batch_size"] = 8
hp["lr"] = 1e-4
hp["num_epochs"] = 500
hp["num_classes"] = 3
hp["class_names"] = ["Benign", "Malignant", "Normal"]

hp["num_layers"] = 12
hp["hidden_dim"] = 768
hp["mlp_dim"] = 2304 #3072
hp["num_heads"] = 6
hp["dropout_rate"] = 0.1

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_data(path, split=0.1):
    images = shuffle(glob(os.path.join(path, "*", "*.jpg")))

    split_size = int(len(images) * split)
    train_x, valid_x = train_test_split(images, test_size=split_size, random_state=42)
    train_x, test_x = train_test_split(train_x, test_size=split_size, random_state=42)

    return train_x, valid_x, test_x

def process_image_label(path):
    """ Reading images """
    path = path.decode()
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (hp["image_size"], hp["image_size"]))
    image = image/255.0

    """ Preprocessing to patches """
    patch_shape = (hp["patch_size"], hp["patch_size"], hp["num_channels"])
    patches = patchify(image, patch_shape, hp["patch_size"])

    # patches = np.reshape(patches, (64, 25, 25, 3))
    # for i in range(64):
    #     cv2.imwrite(f"files/{i}.png", patches[i])

    patches = np.reshape(patches, hp["flat_patches_shape"])
    patches = patches.astype(np.float32)

    """ Label """
    class_name = path.split("/")[-2]
    class_idx = hp["class_names"].index(class_name)
    class_idx = np.array(class_idx, dtype=np.int32)

    return patches, class_idx

def parse(path):
    patches, labels = tf.numpy_function(process_image_label, [path], [tf.float32, tf.int32])
    labels = tf.one_hot(labels, hp["num_classes"])

    patches.set_shape(hp["flat_patches_shape"])
    labels.set_shape(hp["num_classes"])

    return patches, labels

def tf_dataset(images, batch):
    ds = tf.data.Dataset.from_tensor_slices((images))
    ds = ds.map(parse).batch(batch).prefetch(8)
    return ds


if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Directory for storing files """
    create_dir("/content/drive/MyDrive/vit_files2")

    """ Paths """
    dataset_path = "/content/drive/MyDrive/Dataset"
    model_path = os.path.join("/content/drive/MyDrive/vit_files2", "vit_classifier.keras")
    csv_path = os.path.join("/content/drive/MyDrive/vit_files2", "log.csv")

    """ Dataset """
    train_x, valid_x, test_x = load_data(dataset_path)
    print(f"Train: {len(train_x)} - Valid: {len(valid_x)} - Test: {len(test_x)}")

    train_ds = tf_dataset(train_x, batch=hp["batch_size"])
    valid_ds = tf_dataset(valid_x, batch=hp["batch_size"])

    """ Model """
    model = ViT(hp)
    model.compile(
        loss="categorical_crossentropy",
        optimizer=tf.keras.optimizers.Adam(hp["lr"], clipvalue=1.0),
        metrics=["acc"]
    )

    callbacks = [
        ModelCheckpoint(model_path, monitor='val_loss', verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, min_lr=1e-10, verbose=1),
        CSVLogger(csv_path),
        EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=False),
    ]

    model.fit(
        train_ds,
        epochs=hp["num_epochs"],
        validation_data=valid_ds,
        callbacks=callbacks
    )

    ## ...

Train: 879 - Valid: 109 - Test: 109
Epoch 1/500
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 650ms/step - acc: 0.4435 - loss: 2.1738
Epoch 1: val_loss improved from inf to 0.89219, saving model to /content/drive/MyDrive/vit_files2/vit_classifier.keras
110/110 ━━━━━━━━━━━━━━━━━━━━ 190s 1s/step - acc: 0.4436 - loss: 2.1670 - val_acc: 0.5872 - val_loss: 0.8922 - learning_rate: 1.0000e-04
Epoch 2/500
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 299ms/step - acc: 0.4650 - loss: 1.0549
Epoch 2: val_loss did not improve from 0.89219
110/110 ━━━━━━━━━━━━━━━━━━━━ 78s 313ms/step - acc: 0.4649 - loss: 1.0550 - val_acc: 0.5872 - val_loss: 0.9189 - learning_rate: 1.0000e-04
Epoch 3/500
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 323ms/step - acc: 0.4659 - loss: 1.0359
Epoch 3: val_loss did not improve from 0.89219
110/110 ━━━━━━━━━━━━━━━━━━━━ 44s 337ms/step - acc: 0.4658 - loss: 1.0359 - val_acc: 0.5872 - val_loss: 0.9564 - learning_rate: 1.0000e-04
Epoch 4/500
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 328ms/step - acc: 0.4872 - loss: 1.0081
Epo

In [ ]:
import cv2
from patchify import patchify
import numpy as np
hp = {}
hp["image_size"] = 200
hp["num_channels"] = 3
hp["patch_size"] = 25
hp["num_patches"] = (hp["image_size"]**2) // (hp["patch_size"]**2)
hp["flat_patches_shape"] = (hp["num_patches"], hp["patch_size"]*hp["patch_size"]*hp["num_channels"])

hp["batch_size"] = 8
hp["lr"] = 1e-4
hp["num_epochs"] = 500
hp["num_classes"] = 3
hp["class_names"] = ["Benign", "Malignant", "Normal"]

hp["num_layers"] = 12
hp["hidden_dim"] = 768
hp["mlp_dim"] = 2304
hp["num_heads"] = 6
hp["dropout_rate"] = 0.1

path = "/content/drive/MyDrive/Dataset/Malignant/Malignant case (2).jpg"
image = cv2.imread(path, cv2.IMREAD_COLOR)
image = cv2.resize(image, (hp["image_size"], hp["image_size"]))
image = image/255.0

""" Preprocessing to patches """
patch_shape = (hp["patch_size"], hp["patch_size"], 3)
patches = patchify(image, patch_shape, hp["patch_size"])
patches = np.reshape(patches, hp["flat_patches_shape"])
patches = patches.astype(np.float32)
patches = np.expand_dims(patches, axis=0)


(8, 8, 1, 25, 25, 3)
(64, 1875)


In [ ]:

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from patchify import patchify
import tensorflow as tf

""" Hyperparameters """
hp = {}
hp["image_size"] = 200
hp["num_channels"] = 3
hp["patch_size"] = 25
hp["num_patches"] = (hp["image_size"]**2) // (hp["patch_size"]**2)
hp["flat_patches_shape"] = (hp["num_patches"], hp["patch_size"]*hp["patch_size"]*hp["num_channels"])

hp["batch_size"] = 8
hp["lr"] = 1e-4
hp["num_epochs"] = 500
hp["num_classes"] = 3
hp["class_names"] = ["Benign", "Malignant", "Normal"]

hp["num_layers"] = 12
hp["hidden_dim"] = 768
hp["mlp_dim"] = 2304
hp["num_heads"] = 6
hp["dropout_rate"] = 0.1


if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Paths """
    dataset_path = "/content/drive/MyDrive/Dataset"
    model_path = os.path.join("/content/drive/MyDrive/vit_files2", "vit_classifier.keras")

    """ Dataset """
    train_x, valid_x, test_x = load_data(dataset_path)
    print(f"Train: {len(train_x)} - Valid: {len(valid_x)} - Test: {len(test_x)}")

    test_ds = tf_dataset(test_x, batch=hp["batch_size"])

    """ Model """
    model = ViT(hp)
    model.load_weights(model_path)
    model.compile(
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
        optimizer=tf.keras.optimizers.Adam(hp["lr"]),
        metrics=["acc"]
    )

    model.evaluate(test_ds)

Train: 879 - Valid: 109 - Test: 109


KeyboardInterrupt: 

In [ ]:
from patchify import patchify

vit = tf.keras.models.load_model("/content/drive/MyDrive/vit_files2/vit_classifier.keras")

In [ ]:
import cv2
import numpy as np
hp = {}
hp["image_size"] = 200
hp["num_channels"] = 3
hp["patch_size"] = 25
hp["num_patches"] = (hp["image_size"]**2) // (hp["patch_size"]**2)
hp["flat_patches_shape"] = (hp["num_patches"], hp["patch_size"]*hp["patch_size"]*hp["num_channels"])

hp["batch_size"] = 8
hp["lr"] = 1e-4
hp["num_epochs"] = 500
hp["num_classes"] = 3
hp["class_names"] = ["Benign", "Malignant", "Normal"]

hp["num_layers"] = 12
hp["hidden_dim"] = 768
hp["mlp_dim"] = 2304 #3072
hp["num_heads"] = 6
hp["dropout_rate"] = 0.1
path = "/content/drive/MyDrive/Dataset/Malignant/Malignant case (461).jpg"
image = cv2.imread(path, cv2.IMREAD_COLOR)
image = cv2.resize(image, (hp["image_size"], hp["image_size"]))
image = image/255.0

""" Preprocessing to patches """
patch_shape = (hp["patch_size"], hp["patch_size"], hp["num_channels"])
patches = patchify(image, patch_shape, hp["patch_size"])

patches = np.reshape(patches, hp["flat_patches_shape"])
patches = patches.astype(np.float32)
patches = np.expand_dims(patches, axis=0)  # Reshape to (1, 64, 1875)
vit.predict(patches)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


array([[1.2494792e-01, 8.7501913e-01, 3.2904467e-05]], dtype=float32)

In [ ]:
list = [3.4055956e-02, 9.6569252e-01, 2.5156265e-04]
print(list.index(max(list)))

1
